# User Routes

> a functional approach, routes only, to interacting with domo users.


In [1]:
# | default_exp routes.user

In [2]:
# | exporti
from enum import Enum
import httpx

import utils.DictDot as dd
import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

# GET Users


In [3]:
# | export
async def get_all_users(
    auth: dmda.DomoAuth, debug_api: bool = False
) -> rgd.ResponseGetData:

    """retrieves all users from Domo"""
    url = f"https://{ auth.domo_instance}.domo.com/api/content/v2/users"

    return await gd.get_data(url=url, method="GET", auth=auth, debug_api=debug_api)


#### sample implementation of get_all_users


In [4]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_all_users(auth=token_auth)
pd.DataFrame(res.response[0:5])

os.environ["DOMO_DOJO_ACCESS_TOKEN"]

'2f4cb212853732ef796abd46077cf1162c0a3ed7cf1ff079'

# Search Users

This series of functions searchse the v1_users_api. The body of the search api can be retrieved by searching admin in the Domo User interface.


In [5]:
# | export
def generate_search_users_body_by_id(
    user_ids: list[str],  # list of user ids to search
) -> dict:  # dict to pass to search v1_users_search_api
    """search v1_users_search_api"""

    return {
        # "showCount": true,
        # "count": false,
        "includeDeleted": False,
        "includeSupport": False,
        "filters": [
            {"field": "id", "filterType": "value",
                "values": user_ids, "operator": "EQ"}
        ],
    }


In [6]:
# | export
def generate_search_users_body_by_email(
    user_email_ls: list[
        str
    ],  # list of user emails to search.  Note:  search does not appear to be case sensitive
) -> dict:  # dict to pass to search v1_users_search_api
    """search v1_users_search_api"""

    return {
        # "showCount": true,
        # "count": false,
        "includeDeleted": False,
        "includeSupport": False,
        "limit": 200,
        "offset": 0,
        "sort": {"field": "displayName", "order": "ASC"},
        "filters": [
            {
                "filterType": "text",
                "field": "emailAddress",
                "text": " ".join(user_email_ls),
            }
        ],
    }

In [7]:
# | export
def process_v1_search_users(
    v1_user_ls: list[dict],  # list of users from v1_users_search API
) -> list[dict]:  # sanitized list of users.
    """sanitizes the response from v1_users_search API and removes unecessary attributes"""

    clean_users = []

    for obj_user in v1_user_ls:

        dd_user = dd.DictDot(obj_user)

        clean_users.append(
            {
                "id": dd_user.id,
                "displayName": dd_user.displayName,
                "roleId": dd_user.roleId,
                "userName": dd_user.userName,
                "emailAddress": dd_user.emailAddress,
            }
        )

    return clean_users

In [8]:
# | export
class SearchUser_NoResults(de.DomoError):
    def __init__(
        self, domo_instance, function_name="search_users", search_criteria=None
    ):

        search_str = f"- {search_criteria}" if search_criteria else ""

        print(search_str)

        super().__init__(
            message=f"query {search_str} returned no users", function_name=function_name
        )

In [9]:
# | export
async def search_users(
    auth: dmda.DomoAuth, body: dict, debug_api: bool = False, return_raw: bool = False
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/identity/v1/users/search"

    res = await gd.get_data(
        url=url,
        method="POST",
        auth=auth,
        body=body,
        debug_api=debug_api,
    )

    if return_raw:
        return res

    if not res.is_success:
        return res

    if res.is_success and len(res.response.get("users")) == 0:
        raise SearchUser_NoResults(domo_instance=auth.domo_instance)

    res.response = process_v1_search_users(res.response.get("users"))
    return res

#### sample implementation of search_users


In [10]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

search_email_body = generate_search_users_body_by_email(
    user_email_ls=["jae@onyxreporting.com"]
)

res = await search_users(
    auth=token_auth, body=search_email_body, return_raw=False, debug_api=False
)

pd.DataFrame(res.response)

,id,displayName,roleId,userName,emailAddress
0,1893952720,Jae Wilson1,1,jae@onyxreporting.com,jae@onyxreporting.com


# Publish and Virtual Users

> Virtual Users are necessary for assigning PDP policies when publish jobs are created.


In [11]:
# | export
async def search_virtual_user_by_subscriber_instance(
    auth: dmda.DomoAuth,  # domo auth object
    subscriber_instance_ls: list[str],  # list of subscriber domo instances
    debug_api: bool = False,  # debug API requests
) -> rgd.ResponseGetData:  # list of virtual domo users
    """retrieve virtual users for subscriber instances tied to one publisher"""

    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/proxy_user/domain/"

    body = {
        "domains": [
            f"{subscriber_instance}.domo.com"
            for subscriber_instance in subscriber_instance_ls
        ]
    }

    return await gd.get_data(
        url=url,
        method="POST",
        auth=auth,
        body=body,
        debug_api=debug_api,
    )

#### sample of search virtual users by subscriber_instance


In [12]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await search_virtual_user_by_subscriber_instance(
    auth=token_auth, subscriber_instance_ls=["domo-community", "test"], debug_api=False
)


pd.DataFrame(res.response)

,id,publisherDomain,customerId,subscriberDomain,virtualUserId,created,activeSubscriptions
0,44a56146-f422-4175-9c3b-a194f339f9b6,domo-community.domo.com,mmmm-0012-0200,domo-community.domo.com,fc:f230ba95-bc49-4875-a0db-3c7cd58ed3cc,1680594505000,0
1,8b5c26e0-db3a-40f8-855f-6189ac3cf5c8,domo-community.domo.com,mmmm-0012-0200,test.domo.com,fc:e3ce6205-6717-42b4-ba1c-16b5e40eca98,1680594505000,0


# CRUD Routes and User Attributes


In [13]:
# | export
async def create_user(
    auth: dmda.DomoAuth,
    display_name: str,
    email_address: str,
    role_id: int,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/content/v3/users"

    body = {"displayName": display_name,
            "detail": {
                "email": email_address},
            "roleId": role_id
            }

    return await gd.get_data(
        url=url,
        method="POST",
        body=body,
        auth=auth, debug_api=debug_api, session=session
    )


In [24]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

await create_user(auth = token_auth, display_name = 'test_and_delete', email_address = 'test26@test.com', role_id = 5)


ResponseGetData(status=200, response={'id': 746999464, 'invitorUserId': 1893952720, 'displayName': 'test_and_delete', 'role': 'Social', 'roleId': 5, 'detail': {'email': 'test26@test.com', 'pending': True, 'active': True, 'created': 1680664879, 'modified': 1680664879}}, is_success=True)

In [15]:
# | export
async def set_user_landing_page(
    auth: dmda.DomoAuth, user_id: str, page_id: str, debug_api: bool = False
):

    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/landings/target/DESKTOP/entity/PAGE/id/{page_id}/{user_id}"

    return await gd.get_data(
        url=url,
        method="PUT",
        auth=auth,
        # body = body,
        debug_api=debug_api,
    )


In [16]:
# | export
async def reset_password(
    auth: dmda.DomoAuth,
    user_id: str,
    new_password: str,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/identity/v1/password"

    body = {"domoUserId": user_id, "password": new_password}

    return await gd.get_data(
        url=url,
        method="PUT",
        auth=auth,
        body=body,
        debug_api=debug_api,
    )


In [17]:
# | export
async def request_password_reset(
    domo_instance: str, 
    email: str, locale="en-us", debug_api: bool = False,
    session : httpx.AsyncClient = None
):
    url = f"https://{domo_instance}.domo.com/api/domoweb/auth/sendReset"

    params = {"email": email, "local": locale}

    return await gd.get_data(
        url=url, method="GET", params=params, auth=None, debug_api=debug_api
    )


### Properties you can PATCH


In [18]:
# | export
class UserProperty_Type(Enum):
    display_name = "displayName"
    email_address = "emailAddress"
    phone_number = "phoneNumber"
    title = "title"
    department = "department"
    web_landing_page = "webLandingPage"
    web_mobile_landing_page = "webMobileLandingPage"
    role_id = "roleId"
    employee_id = "employeeId"
    employee_number = "employeeNumber"
    hire_date = "hireDate"
    reports_to = "reportsTo"


class UserProperty:
    property_type: UserProperty_Type
    values: str

    def __init__(self, property_type: UserProperty_Type, values):
        self.property_type = property_type
        self.values = self._valid_value(values)

    @staticmethod
    def _valid_value(values):
        return values if isinstance(values, list) else [values]

    def to_json(self):
        return {
            "key": self.property_type.value,
            "values": self._valid_value(self.values),
        }

In [19]:
# | export
def generate_patch_user_property_body(user_property_ls: [UserProperty]):
    return {
        "attributes": [user_property.to_json() for user_property in user_property_ls]
    }

#### sample implementation of generate_user_property_body


In [20]:
user_property_ls = [
    UserProperty(UserProperty_Type.display_name, "jae myong wilson"),
    UserProperty(UserProperty_Type.email_address, "jae@onyxreporting.com"),
    UserProperty(UserProperty_Type.role_id, 1),
]

generate_patch_user_property_body(user_property_ls)

{'attributes': [{'key': 'displayName', 'values': ['jae myong wilson']},
  {'key': 'emailAddress', 'values': ['jae@onyxreporting.com']},
  {'key': 'roleId', 'values': [1]}]}

In [21]:
# | export
async def update_user(
    user_id: str,
    user_property_ls: [UserProperty],
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):
    url = f"https://{auth.domo_instance}.domo.com/api/identity/v1/users/{user_id}"

    body = (
        generate_patch_user_property_body(user_property_ls)
        if isinstance(user_property_ls[0], UserProperty)
        else user_property_ls
    )

    return await gd.get_data(
        url=url,
        method="Patch",
        auth=auth,
        body=body,
        debug_api=debug_api,
        session=session,
    )

In [22]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

user_property_ls = [
    UserProperty(UserProperty_Type.display_name, "test 3"),
    UserProperty(UserProperty_Type.email_address, "test3@test.com"),
    UserProperty(UserProperty_Type.role_id, 810756122),
]

await update_user(
    user_id=1681443709, user_property_ls=user_property_ls, auth=token_auth
)

ResponseGetData(status=200, response={'attributes': [{'key': 'id', 'values': [1681443709]}, {'key': 'displayName', 'values': ['test 3']}, {'key': 'userName', 'values': ['test3@test.com']}, {'key': 'emailAddress', 'values': ['test3@test.com']}, {'key': 'modified', 'values': [1680664869762]}, {'key': 'created', 'values': [1664936244000]}, {'key': 'roleId', 'values': [810756122]}, {'key': 'isAnonymous', 'values': [True]}, {'key': 'isSystemUser', 'values': [False]}, {'key': 'isPending', 'values': [True]}, {'key': 'isActive', 'values': [True]}, {'key': 'invitorUserId', 'values': [1893952720]}, {'key': 'avatarKey', 'values': ['/api/content/v1/avatar/USER/1681443709']}], 'id': 1681443709, 'displayName': 'test 3', 'roleId': 810756122, 'userName': 'test3@test.com', 'emailAddress': 'test3@test.com'}, is_success=True)

In [23]:
# | hide
import nbdev

nbdev.nbdev_export()